In [ ]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
from pandas import read_excel
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta


In [ ]:
url = "https://music.bugs.co.kr/chart/track/day/total"

userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"

session = requests.Session()

session.headers.update({
        "Referer": "",
        "User-Agent": userAgent,
})

r = session.get(url)

if r.status_code != 200:
    msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
    raise Exception(msg)

r.encoding = "utf-8"
soup = BeautifulSoup(r.content,"html.parser")


In [ ]:
일간차트 = []
query = datetime(2023,5,5)
now = datetime.now()
nowday = now.strftime('%Y%m%d')
days = int(nowday)

while query <= datetime.strptime(nowday, '%Y%m%d'):
    url = "https://music.bugs.co.kr/chart/track/day/total?chartdate={}".format(query.strftime('%Y%m%d'))
    r = session.get(url)
    soup = BeautifulSoup(r.content,"html.parser")
    titles = soup.select("p.title")
    artists = soup.select("p.artist")
    albums = soup.select("a.album")

    for i in range(len(titles)):
        title = titles[i].text.strip()
        artist = artists[i].text.strip()
        album = albums[i+1].text.strip()
        rank = i+1 
        if rank ==1:
            days-=1

        chart_list = {"곡명": title, "아티스트":artist, "앨범명":album, "순위" : rank, "날짜": query.strftime('%Y%m%d')}
        일간차트.append(chart_list)
    query += timedelta(days=1)
df = DataFrame(일간차트)
df.set_index("순위", inplace=True)
df.to_excel("일간차트.xlsx")
df